In [13]:
debug_mode = False

In [8]:
import spacy
import json
from ipynb.fs.defs.fonctions_preprocess import open_file
from ipynb.fs.defs.fonctions_preprocess import serialisation_data

In [2]:
data = open_file("nouns.json")

print(len(data))

13803483


In [11]:
if debug_mode:
    test = data[0:200]

In [6]:
def list_into_string(tuple_list):

    extract = []
    extract_s = ""

    #Permet d'enlever les doublons
    for tup in tuple_list:
        if tup[0] not in extract:
            extract.append(tup[0])
            
    print("Taille des occurences uniques : ", len(extract))
        
    for word in extract:
        extract_s += word
        extract_s += " "
        
    return extract_s

In [7]:
corpus = list_into_string(data)    

Taille des occurences uniques :  132725


In [9]:
serialisation_data (corpus, "corpus_string.json")

In [10]:
nlp = spacy.load("fr_core_news_md")

In [11]:
nlp.max_length = 1500000

doc1 = nlp(corpus)
doc2 = nlp(corpus)

In [12]:
def calculate_similarity (token1, token2):
    """Calculate Similarity between two tokens using SpaCy pre-trained model
    -> Two tokens
    <- similarity score between these two tokens"""

    similarity = token1.similarity(token2)
    
    if similarity >= 0.5 and similarity != 1 : return (token1, token2, similarity) 

In [13]:
def construction_liste_similarity(doc1, doc2):
    """Builds list of similarities. One element of the list is built so : (token1, token2, similarity)
    -> doc1 : list of words
    -> doc2 : other list of words
    <- list of tuples made of two words and their similarities
    """
    
    
    
    similarities = []
    
    for i in range (len(doc1)):
        if doc1[i].has_vector != False: tok = doc1[i]
        for k in range(len(doc2)):
            if doc2[k].has_vector != False : 
                similarity = calculate_similarity(tok, doc2[k])
                if similarity != None : similarities.append(similarity)
                    
    return similarities
    

In [14]:
def extract_unique_referents(doc1, doc2):
    """extract unique_referent using word_similarities
    -> string of words
    <- list of words and dictionnary of similarities
    doc_sim = {word1: [(word, similarity), ..., (word, similarity)], word2 : [(word, similarity), ... (word, similarity)], ... wordn : [(word, similarity), (word, similarity)]}
    
    We use word similarities using spacy module. We take similarities >= 0.5 to delete low similarities and "clean" corpus) 
    """
    
    humans_referent = []
    doc_sim = {}

    similarities = construction_liste_similarity(doc1, doc2)

    print("Similarities are done !")
    
    for sim in similarities:
        if sim[1] not in humans_referent: humans_referent.append((sim[1])) 
        if sim[0] not in doc_sim : 
            doc_sim[sim[0]] = []
        doc_sim[sim[0]].append((sim[1], sim[2]))

    print("Taillle de humans_referent : ", len(humans_referent), " et taille de doc_sim : ", len(doc_sim))
    return (humans_referent, doc_sim)

In [15]:
extraction = extract_unique_referents(doc1, doc2)

KeyboardInterrupt: 

In [ ]:
if debug_mode:
    for dic in extraction[1]:
        print("dic :", dic, " -> ", extraction[1][dic])
        print()